In [2]:
import copy
import importlib
import os
import pathlib
import shutil

import prompt
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import (
    WatsonxLLM,
)

from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams


load_dotenv()


model = Model(
    model_id=ModelTypes.FLAN_T5_XXL,
    credentials={
        "apikey": os.getenv("IBM_API_KEY"),
        "url": "https://us-south.ml.cloud.ibm.com",
    },
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MAX_NEW_TOKENS: 100,
        GenParams.TEMPERATURE: 0,
        GenParams.RANDOM_SEED: 12345,
    },
    project_id=os.getenv("PROJECT_ID"),
)

llm = WatsonxLLM(model=model)

print(llm("hello how are you?"))

i am fine


In [3]:
def build_prompt(question):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    BOS, EOS = "<s>", "</s>"
    DEFAULT_SYSTEM_PROMPT = """
    You are an expert Q&A system that is trusted around the world. Always answer the query using the provided context information, and not prior knowledge.
    Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
    """
    prompt = "".join([B_SYS, DEFAULT_SYSTEM_PROMPT, E_SYS, question]).strip()
    return f"{BOS}{B_INST} {prompt} {E_INST}"

ROUTING_TEMPLATE = """
Classify the following question into one of these options. 

1) Property specific transactional query
2) General transactional query
3) Property specific general query
4) General query

Query: Still got available units for Residensi Zig?

Answer: 1

Query: what is a good house that is less then 900k?

Answer: 2

Query: any property in Mont Kiara?

Answer: 3

Query: what are the amenities in the minh?

Answer: 3

Question: {{question}} 

Answer:
"""

In [28]:
questions = [
    {"question": "what are the layout types for Zig?", "answer": 1},
    {"question": "what are available units for the minh?", "answer": 1},
    {"question": "How many units that have the status Hold for connaught", "answer": 1},
]

generated_answers = [
    llm(ROUTING_TEMPLATE.replace("{{question}}", x["question"])) for x in questions
]

df = pd.DataFrame(questions)
df["generated_answer"] = generated_answers
df["generated_answer"] = df["generated_answer"].astype(int)
print("Accuracy: {:.2f}".format((df["answer"] == df["generated_answer"]).sum() / df.shape[0]))
df

Accuracy: 0.33


,question,answer,generated_answer
0,what are the layout types for Zig?,1,3
1,what are available units for the minh?,1,1
2,How many units that have the status Hold for c...,1,3
